In [44]:
import pyvisa
import numpy as np
import time
import random

In [32]:
rm = pyvisa.ResourceManager('@py')
print(rm)
rm.list_resources()

Resource Manager of Visa Library at py


('ASRL/dev/ttyS31::INSTR',
 'ASRL/dev/ttyS30::INSTR',
 'ASRL/dev/ttyS29::INSTR',
 'ASRL/dev/ttyS28::INSTR',
 'ASRL/dev/ttyS27::INSTR',
 'ASRL/dev/ttyS26::INSTR',
 'ASRL/dev/ttyS25::INSTR',
 'ASRL/dev/ttyS24::INSTR',
 'ASRL/dev/ttyS23::INSTR',
 'ASRL/dev/ttyS22::INSTR',
 'ASRL/dev/ttyS21::INSTR',
 'ASRL/dev/ttyS20::INSTR',
 'ASRL/dev/ttyS19::INSTR',
 'ASRL/dev/ttyS18::INSTR',
 'ASRL/dev/ttyS17::INSTR',
 'ASRL/dev/ttyS16::INSTR',
 'ASRL/dev/ttyS15::INSTR',
 'ASRL/dev/ttyS14::INSTR',
 'ASRL/dev/ttyS13::INSTR',
 'ASRL/dev/ttyS12::INSTR',
 'ASRL/dev/ttyS11::INSTR',
 'ASRL/dev/ttyS10::INSTR',
 'ASRL/dev/ttyS9::INSTR',
 'ASRL/dev/ttyS8::INSTR',
 'ASRL/dev/ttyS7::INSTR',
 'ASRL/dev/ttyS6::INSTR',
 'ASRL/dev/ttyS5::INSTR',
 'ASRL/dev/ttyS4::INSTR',
 'ASRL/dev/ttyS3::INSTR',
 'ASRL/dev/ttyS2::INSTR',
 'ASRL/dev/ttyS1::INSTR',
 'ASRL/dev/ttyS0::INSTR',
 'ASRL/dev/ttyACM1::INSTR')

In [33]:
inst = rm.open_resource('ASRL/dev/ttyACM1::INSTR')
print(inst)

SerialInstrument at ASRL/dev/ttyACM1::INSTR


In [34]:
values = [123,2351]
inst.write_ascii_values('', values)
#inst.write_ascii_values('', values, converter='f', separator=',') # does the same thing as above

24

In [35]:
inst.write('264,3219\n')

11

In [35]:
measurements = inst.read_ascii_values()
print(measurements)

[824.0, 803.0]


In [17]:
measurements = inst.read_ascii_values(container=np.array)
print(measurements.dtype)
print(measurements)

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [49]:
print(inst.read())

123,2351



In [45]:
# Test how fast this thing can loop
# Fixed number of iterations
t0 = time.time()
iterations = 1000
for i in range(iterations):
    values = [0,0]
    values[0] = random.randint(0,4095)
    values[1] = random.randint(0,4095)
    inst.write_ascii_values('', values)
    measurements = inst.read_ascii_values()
    #print(measurements)

t1 = time.time()
total = t1-t0
print(iterations, "iterations took", total, "s.")
print(iterations / total, "iterations/s.")


1000 iterations took 0.5769174098968506 s.
1733.3503597660435 iterations/s.
